In [1]:
import tarfile
import os
import glob
import re
import shutil
import pandas as pd
import numpy as np

In [2]:
path = "C:/Users/domanski/Downloads/kubamarv/"
with open("ASIB.3ABIC.arch") as f: lineList = f.readlines()
arch = [item.split() for item in lineList]
acro = {"FSIP1": "ASIB","FBIP1": "ABIC1","FBIP2": "ABIC2","FBIP3": "ABIC3"}

In [3]:
# stripping counters from file
# df_size = len(df.index)
df = pd.DataFrame()

for file in glob.glob(path+'*_TESTS.TS0.TC9_SUD.BBU.SM1.*.info'):
    round = file.split('_')[-3]
    round = round.split('\\')
    if 'XFI' in round: 
        round = round[1]
    else:
        round = round[0]
#     df[round+"tx"] =np.nan
#     df[round+"rx"] =np.nan
    with open(file) as f: lineList = f.readlines()
    flag = 0
    for index,line in enumerate(lineList):
        if "TX_OCTETS" in line:
            board = line.split("SUB1.")[1][:5]
            txo = line.split("TX_OCTETS")[1]
            xgm = lineList[index-1].split("XGMAC")[1]
            xgm = xgm.strip('\n')
            txo = txo.strip('\n')
            if acro[board]=="ASIB":
                device = ".AXM.XGMAC"
            else:
                device = ".LFA.XGMAC"
            df.loc[acro[board]+device+xgm+" tx",round]=int(txo[3:],16)
        if "RX_OCTETS" in line:
            board = line.split("SUB1.")[1][:5]
            txo = line.split("RX_OCTETS")[1]
            xgm = lineList[index-1].split("XGMAC")[1]
            xgm = xgm.strip('\n')
            txo = txo.strip('\n')
            if acro[board]=="ASIB":
                device = ".AXM.XGMAC"
            else:
                device = ".LFA.XGMAC"
            df.loc[acro[board]+device+xgm+" rx",round]=int(txo[3:],16)
        if "UC Sent" in line:
            i=index+3
            while "/" in lineList[i]:
                mar = lineList[i].split("INFO")
                mar = mar[1].split()
                df.loc[acro[board]+".MARVEL."+mar[0]+" tx",round]=int(mar[4])
                i+=1
        if "UC Received" in line:
            i=index+3
            while "/" in lineList[i]:
                mar = lineList[i].split("INFO")
                mar = mar[1].split()
                df.loc[acro[board]+".MARVEL."+mar[0]+" rx",round]=int(mar[4])
                i+=1

print(df)

                                11            13            14            15  \
ASIB.AXM.XGMAC0 tx    2.185907e+10  2.185907e+10  1.391671e+10  2.185907e+10   
ASIB.AXM.XGMAC0 rx    2.185907e+10  2.185907e+10  2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC16 tx   2.185907e+10  2.185907e+10  1.356362e+10  2.185907e+10   
ASIB.AXM.XGMAC16 rx   2.185907e+10  2.185907e+10  2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC96 tx   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
ASIB.AXM.XGMAC96 rx   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
ASIB.AXM.XGMAC97 tx   2.185907e+10  2.185907e+10  2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC97 rx   2.185907e+10  2.185907e+10  5.258692e+10  2.185907e+10   
ASIB.AXM.XGMAC112 tx  2.185907e+10  2.185907e+10  2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC112 rx  2.185907e+10  2.185907e+10  1.356362e+10  2.185907e+10   
ASIB.AXM.XGMAC113 tx  1.340800e+04  1.327600e+04  1.280200e+04  1.277200e+04   
ASIB.AXM.XGMAC113 rx  4.465600e+04  4.48

In [13]:
arch_df = pd.DataFrame(arch)
a = arch_df[0].astype(str)
b = arch_df[1].astype(str)
arch_df[0] = a + ' tx'
arch_df[1] = b + ' rx'
arch_df[2] = a + ' rx'
arch_df[3] = b + ' tx'
arch_df = arch_df.drop([5])
print(arch_df)

                       0                     1                     2  \
0           ASIB.EIF1 tx    ASIB.AXM.XGMAC0 rx          ASIB.EIF1 rx   
1           ASIB.EIF2 tx   ASIB.AXM.XGMAC16 rx          ASIB.EIF2 rx   
2   ASIB.AXM.XGMAC113 tx    ASIB.MARVEL.0/0 rx  ASIB.AXM.XGMAC113 rx   
3   ASIB.AXM.XGMAC112 tx    ASIB.MARVEL.0/1 rx  ASIB.AXM.XGMAC112 rx   
4    ASIB.AXM.XGMAC97 tx    ASIB.MARVEL.0/2 rx   ASIB.AXM.XGMAC97 rx   
6    ASIB.MARVEL.M0/4 tx  ABIC1.LFA.XGMAC48 rx   ASIB.MARVEL.M0/4 rx   
7    ASIB.MARVEL.M0/5 tx  ABIC1.LFA.XGMAC49 rx   ASIB.MARVEL.M0/5 rx   
8    ASIB.MARVEL.M0/6 tx  ABIC1.LFA.XGMAC50 rx   ASIB.MARVEL.M0/6 rx   
9    ASIB.MARVEL.M0/7 tx  ABIC1.LFA.XGMAC51 rx   ASIB.MARVEL.M0/7 rx   
10   ASIB.MARVEL.M0/8 tx  ABIC2.LFA.XGMAC48 rx   ASIB.MARVEL.M0/8 rx   
11   ASIB.MARVEL.M0/9 tx  ABIC2.LFA.XGMAC49 rx   ASIB.MARVEL.M0/9 rx   
12  ASIB.MARVEL.M0/10 tx  ABIC2.LFA.XGMAC50 rx  ASIB.MARVEL.M0/10 rx   
13  ASIB.MARVEL.M0/11 tx  ABIC2.LFA.XGMAC51 rx  ASIB.MARVEL.M0/1

In [26]:
drop_df = pd.DataFrame()
drop_df["part"]=arch_df[0]+" => "+arch_df[1]
drop_df.set_index("part")
for index, row in arch_df.iterrows():
    if arch_df.loc[index,1] in df.index:
        if arch_df.loc[index,0] in df.index:
            df.loc[arch_df.loc[index,0]+" => "+arch_df.loc[index,1]] = df.loc[arch_df.loc[index,1]]-df.loc[arch_df.loc[index,0]]
            df.loc[arch_df.loc[index,3]+" => "+arch_df.loc[index,2]] = df.loc[arch_df.loc[index,2]]-df.loc[arch_df.loc[index,3]]
            df.loc[arch_df.loc[index,1]+" => "+arch_df.loc[index,3]] = df.loc[arch_df.loc[index,1]]-df.loc[arch_df.loc[index,3]]
            df.loc[arch_df.loc[index,0]+" => "+arch_df.loc[index,2]] = df.loc[arch_df.loc[index,0]]-df.loc[arch_df.loc[index,2]]

In [27]:
pdf = df
for index, row in pdf.iterrows():
    if ">" not in row.name: pdf = pdf.drop(row.name)
print(pdf)

                                                   11       13            14  \
ASIB.AXM.XGMAC113 tx => ASIB.MARVEL.0/0 rx      738.0    738.0  7.380000e+02   
ASIB.AXM.XGMAC112 tx => ASIB.MARVEL.0/1 rx      188.0    438.0  5.320000e+02   
ASIB.AXM.XGMAC97 tx => ASIB.MARVEL.0/2 rx        94.0     94.0  9.400000e+01   
ASIB.AXM.XGMAC113 rx => ASIB.MARVEL.0/0 tx        0.0      0.0  0.000000e+00   
ASIB.AXM.XGMAC112 rx => ASIB.MARVEL.0/1 tx        0.0      0.0  0.000000e+00   
ASIB.AXM.XGMAC97 rx => ASIB.MARVEL.0/2 tx         0.0      0.0  0.000000e+00   
ASIB.MARVEL.0/0 tx => ASIB.AXM.XGMAC113 rx    -1152.0  -1152.0 -1.374000e+03   
ASIB.MARVEL.0/0 rx => ASIB.MARVEL.0/0 tx     -31662.0 -32020.0 -3.462200e+04   
ASIB.MARVEL.0/1 tx => ASIB.AXM.XGMAC112 rx        0.0      0.0  0.000000e+00   
ASIB.MARVEL.0/1 rx => ASIB.MARVEL.0/1 tx        828.0    828.0  8.295458e+09   
ASIB.MARVEL.0/2 tx => ASIB.AXM.XGMAC97 rx         0.0      0.0 -1.691418e+10   
ASIB.MARVEL.0/2 rx => ASIB.MARVEL.0/2 tx

In [28]:
pdf[pdf>100000].dropna(how="all").dropna(axis="columns",how="all")

,14,17,19,21,2,39,44,49,54,56,64,67,68,70,71,77
ASIB.MARVEL.0/1 rx => ASIB.MARVEL.0/1 tx,8.295458e+09,NaN,1.269056e+09,NaN,8.242805e+09,8.140355e+09,8.908646e+09,7.746134e+09,7.894972e+09,8.027987e+09,822025284.0,7.845037e+09,7.765180e+09,8.924068e+09,7.950228e+09,8.242824e+09
ASIB.MARVEL.0/2 rx => ASIB.MARVEL.0/2 tx,NaN,39072436.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ASIB.AXM.XGMAC113 tx => ASIB.AXM.XGMAC113 rx,NaN,NaN,NaN,480372544.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ASIB.AXM.XGMAC112 tx => ASIB.AXM.XGMAC112 rx,8.295458e+09,NaN,1.269056e+09,NaN,8.242805e+09,8.140355e+09,8.908646e+09,7.746133e+09,7.894972e+09,8.027987e+09,822025108.0,7.845037e+09,7.765180e+09,8.924068e+09,7.950228e+09,8.242824e+09
ASIB.AXM.XGMAC97 tx => ASIB.AXM.XGMAC97 rx,NaN,39072248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
print(df)

                                                        11            13  \
ASIB.AXM.XGMAC0 tx                            2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC0 rx                            2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC16 tx                           2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC16 rx                           2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC96 tx                           0.000000e+00  0.000000e+00   
ASIB.AXM.XGMAC96 rx                           0.000000e+00  0.000000e+00   
ASIB.AXM.XGMAC97 tx                           2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC97 rx                           2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC112 tx                          2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC112 rx                          2.185907e+10  2.185907e+10   
ASIB.AXM.XGMAC113 tx                          1.340800e+04  1.327600e+04   
ASIB.AXM.XGMAC113 rx                          4.465600e+04  4.488200e+04   
ABIC1.LFA.XG